In [70]:
import requests
import bs4
from bs4 import BeautifulSoup as soup
import json
import pandas as pd
from pandas import DataFrame

import spacy
from spacy import displacy
from spacy.matcher import Matcher
from spacy.tokens import Span
nlp = spacy.load('en_core_web_sm')
from string import punctuation

def get_ld_json(url: str) -> dict:
    parser = "html.parser"
    req = requests.get(url)
    page = soup(req.text, parser)
    return json.loads("".join(page.find("script", {"type":"application/ld+json"}).contents))

url1 = "https://www.allrecipes.com/recipe/232549"
url2 = "https://www.allrecipes.com/recipe/267555/"
url3 = "https://www.allrecipes.com/recipe/78299/"
url4 = "https://www.allrecipes.com/recipe/13062/"

def get_ingredientScript(url):
    jsonld = get_ld_json(url)
    useful = jsonld[1]
    ingredients = useful["recipeIngredient"]
    return ingredients

In [103]:
def parse_ingredients(url):
    ingredients = get_ingredientScript(url)
    df = DataFrame (ingredients,columns=['ingredients'])
    # 2 cases
    # "number" ("()") ("unit") (adjective) "noun/subject - ingredient" (, other)
    # contains "to taste"
    df = df["ingredients"]
    df_taste = df[df.str.contains('to taste', case = False)]
    
    df_unit = df[~df.str.contains('to taste', case = False)]
    
    # array of arrays: each array is amount, unit, ingredient, descriptor, preparation
    ingredients_parsed = []
    
    for i in df:
        curr_arr = ["", "", "", "", ""]
        print(i)
        if 'to taste' not in i:
        
            # before we look at POS, remove everything after the comma and put it in prep
            split_string = i.split(", ", 1)
            root_phrase = split_string[0]
            if len(split_string) > 1:
                other_piece = split_string[1]
            else:
                other_piece = ""

            curr_arr[4] = other_piece

            split_string2 = root_phrase.split("(", 1)
            if len(split_string2) > 1:
                split_string3 = split_string2[1].split(")", 1)
                curr_arr[3] = split_string3[0]
                root_phrase = split_string2[0].strip() + split_string3[1]
            doc = nlp(root_phrase)
            index = 0
            for token in doc:
                found_num = False
                # only get first number if it matches criteria
                #if found_num == False and token.pos_ == "NOUN" and not token.is_alpha or token.pos_ == "NUM":
                if index == 0:
                    curr_arr[0] = token.text
                elif index == 1:
                    if token.pos_ != "ADJ":
                        curr_arr[1] = token.text
                elif token.dep_ == "ROOT":
                    curr_arr[2] = token.text
                else: 
                    curr_arr[3] = curr_arr[3] + " " + token.text
                index+=1
        else:
            i = i.replace("to taste", "")
            doc = nlp(i)
            curr_arr[0] = "to taste"
            for token in doc:
                if token.dep_ == "ROOT":
                    curr_arr[2] = token.text
                else:
                    curr_arr[3] = curr_arr[3] + " " + token.text
        ingredients_parsed.append(curr_arr)
        #print(token.text, token.pos_, token.tag_, token.dep_, token.shape_, token.is_alpha, token.is_stop, token.children, token.head)
        #displacy.render(doc, style="dep") # change to serve when we go to python


    
    return ingredients_parsed

In [105]:
urls = [url1, url2, url3, url4]
for url in urls:
    print(parse_ingredients(url))

¼ cup butter
½ small white onion, finely diced
3 small zucchinis, diced
3 ears corn, husks and silk removed
sea salt to taste
freshly ground black pepper to taste
[['¼', 'cup', 'butter', '', ''], ['½', '', 'onion', ' white', 'finely diced'], ['3', '', 'zucchinis', '', 'diced'], ['3', 'ears', 'corn', '', 'husks and silk removed'], ['to taste', '', 'salt', ' sea', ''], ['to taste', '', 'pepper', ' freshly ground black', '']]
2 large eggs
⅓ cup milk
1 tablespoon butter, melted
1 teaspoon vanilla extract
1 teaspoon ground cinnamon
4 slices day-old bread, cut into thirds
1 teaspoon confectioners' sugar, or to taste
[['2', '', 'eggs', '', ''], ['⅓', 'cup', 'milk', '', ''], ['1', 'tablespoon', 'butter', '', 'melted'], ['1', 'teaspoon', 'extract', ' vanilla', ''], ['1', 'teaspoon', 'cinnamon', ' ground', ''], ['4', 'slices', 'bread', ' day - old', 'cut into thirds'], ['to taste', '', 'sugar', " 1 teaspoon confectioners ' , or", '']]
2 pounds ground beef chuck
1 pound bulk Italian sausage
3 (15